# ATSP Environment

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(2*"../")

import torch
from collections import defaultdict
from typing import Optional

from rl4co.envs import TSPEnv, ATSPEnv 


In [2]:
env = ATSPEnv(num_loc=10)
env.reset()

TensorDict(
    fields={
        action_mask: Tensor(shape=torch.Size([1, 10]), device=cpu, dtype=torch.bool, is_shared=False),
        current_node: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.int64, is_shared=False),
        done: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
        first_node: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.int64, is_shared=False),
        i: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.int64, is_shared=False),
        observation: Tensor(shape=torch.Size([10, 10]), device=cpu, dtype=torch.float32, is_shared=False),
        reward: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([]),
    device=cpu,
    is_shared=False)

In [6]:
# set logging level
import logging
logging.basicConfig(level=logging.INFO)

In [7]:
# actions = torch.randperm(10)
actions = torch.stack([torch.randperm(10) for _ in range(2)])

td = env.reset(batch_size=[2])

print(actions.shape)
print(td['observation'].shape)
for i in range(10):
    # print(actions[:, i][:, None].shape)
    td.set("action", actions[:, i][:, None])
    td = env.step(td)

# print(td['observation'])
# print(actions)

# td = td.unsqueeze(0)
# actions = actions.unsqueeze(0)
reward = env.get_reward(td, actions)
# print(reward.shape)
print(reward)
# print(reward[:, :, -1])

INFO:rl4co.envs.atsp:Using TMAT class (triangle inequality): True


torch.Size([2, 10])
torch.Size([2, 10, 10])
tensor([2.7388, 2.1939])


In [4]:
import torch

# Create a batch of 2 matrices, each of size 10x10
batch_size = 2
matrix = torch.randn((batch_size, 10, 10))

# Create tensors with the row and column indices to select for each matrix in the batch
idx1 = torch.tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 
                     [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]])
idx2 = torch.tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 
                     [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]])

# Select the elements corresponding to the tensor indices for each matrix in the batch
elements = matrix[torch.arange(batch_size).unsqueeze(1), idx1, idx2]

print(elements.shape) 

torch.Size([2, 10])


In [5]:
distance_matrix = td["observation"]
# assert (
#     torch.arange(actions.size(1), out=actions.data.new())
#     .view(1, -1)
#     .expand_as(actions)
#     == actions.data.sort(1)[0]
# ).all(), "Invalid tour"

# Get indexes of tour. Actions: [batch_size, num_loc]
nodes_src = actions
nodes_tgt = torch.roll(actions, 1, dims=1)

print(nodes_src)
print(nodes_tgt)

# Get distances of tour
distance_matrix[:, nodes_src, nodes_tgt].sum(-1)


tensor([[9, 5, 6, 1, 8, 3, 7, 4, 0, 2],
        [0, 1, 6, 8, 4, 9, 3, 7, 5, 2]])
tensor([[2, 9, 5, 6, 1, 8, 3, 7, 4, 0],
        [2, 0, 1, 6, 8, 4, 9, 3, 7, 5]])


tensor([[2.6223, 2.7526],
        [3.5142, 3.4364]])